### 500543568_530454874_TransferLearning

Team members: Andrew Zhang (SID:500543568) Vincent Yunansan (SID:530454874)  

This file generates output for transfer learning for a combination of pre-trained models and optimizers.

Note this file was run on GPU: mps, please adjust device allocation to run on cuda.

In [2]:
#EDA imports
import logging

#basic libraries
import numpy as np
import matplotlib.pyplot as plt 
import pandas as pd

#load and counts and scores 
import os
import re
from io import StringIO
from collections import Counter
import time
from datetime import datetime
import glob
from sklearn.metrics import f1_score

#nltk for word counts in caption
import nltk
from nltk.corpus import stopwords

#images
from IPython.display import display, Image as DisplayImage
from PIL import Image

#torch
import torch
from torch.utils.data import random_split, Dataset, DataLoader
from torchvision import transforms, models
import torch.optim as optim

from sklearn.model_selection import train_test_split

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# read train file
FILENAME = 'data/train.csv'
with open(FILENAME) as file:
    lines = [re.sub(r'([^,])"(\s*[^\n])', r'\1/"\2', line) for line in file]
    train_data = pd.read_csv(StringIO(''.join(lines)), escapechar="/")

FILENAME = 'data/test.csv'
with open(FILENAME) as file:
    lines = [re.sub(r'([^,])"(\s*[^\n])', r'\1/"\2', line) for line in file]
    test_data = pd.read_csv(StringIO(''.join(lines)), escapechar="/")

In [5]:
def one_hot_encode_multilabels(df, label_column):
    """
    One-hot encode a column of labels separated by spaces in a DataFrame.

    input:
    df (pd.DataFrame): The DataFrame containing the data.
    label_column (str): The name of the column containing the space-separated labels.

    output:
    pd.DataFrame: A DataFrame with the original columns plus the one-hot encoded labels.
    """
    # Check if the label column exists in DataFrame
    if label_column not in df.columns:
        raise ValueError(f"The specified column '{label_column}' does not exist in the DataFrame.")

    # Split the labels by space and apply one-hot encoding
    # We create a Series of lists of labels, then explode it to have a single label per row
    s = df[label_column].str.split().explode()

    # Convert the exploded labels into integers for proper one-hot encoding
    s = s.astype(int)

    # Get dummies and then sum back to original DataFrame shape
    dummies = pd.get_dummies(s, prefix='Label').groupby(level=0).sum()

    # Concatenate the original DataFrame with the new one-hot encoded columns
    df_encoded = pd.concat([df, dummies], axis=1)

    # Print counts for each label
    label_counts = dummies.sum()
    return df_encoded

class EarlyStopper:
    '''
    Early stopping class to stop training if the validation loss does not improve after a certain number of epochs.
    '''
    def __init__(self, patience=4, min_delta=0):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.min_validation_loss = float('inf')

    def early_stop(self, validation_loss):
        if validation_loss < self.min_validation_loss:
            self.min_validation_loss = validation_loss
            self.counter = 0
        elif validation_loss > (self.min_validation_loss + self.min_delta):
            self.counter += 1
            if self.counter >= self.patience:
                return True
        return False
    
# for train and val data
class ImageDataset(Dataset):
    """
    Dataset object for train image data (since it has both image and label)
    input:
        dataframe (pd.DataFrame): DataFrame containing the image paths and one-hot encoded labels.
        image_dir (str): Directory where images are stored.
        transform (callable, optional): Optional transform to be applied on a sample.
    """
    def __init__(self, dataframe, image_dir, transform=None):
        self.dataframe = dataframe
        self.image_dir = image_dir
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        image_path = os.path.join(self.image_dir, self.dataframe.iloc[idx, 0])
        image = Image.open(image_path).convert('RGB')
        
        if self.transform:
            image = self.transform(image)

        label = self.dataframe.iloc[idx, 1:].values.astype('float32') 
        
        return image, label 


#for test data
class ImageDataset2(Dataset):
    """
    Dataset object for test image data (since it only has image and no label)
    input:
        dataframe (pd.DataFrame): DataFrame containing the image paths and one-hot encoded labels.
        image_dir (str): Directory where images are stored.
        transform (callable, optional): Optional transform to be applied on a sample.
    """
    def __init__(self, dataframe, image_dir, transform=None):
        self.dataframe = dataframe
        self.image_dir = image_dir
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        img_name = os.path.join(self.image_dir, self.dataframe.iloc[idx, 0])
        image = Image.open(img_name).convert('RGB')
        if self.transform:
            image = self.transform(image)
        return image
    
#logger

class Logger:
    '''
    Generate log files for analysis. This object runs in the background without user input.
    '''
    def __init__(self, model_name, timestamp, logging_path='models/logs/'):
        self.model_name = model_name
        self.timestamp = timestamp
        self.logging_path = logging_path
        os.makedirs(self.logging_path, exist_ok=True)

        # Create a custom logger
        self.logger = logging.getLogger(__name__)
        self.logger.handlers = []  # Clear existing handlers, otherwise we'll get multiple as has happened
        self.logger.setLevel(logging.INFO)

        # Create handlers
        c_handler = logging.StreamHandler()
        f_handler = logging.FileHandler(f'{self.logging_path}{self.model_name}.log')
        c_handler.setLevel(logging.INFO)
        f_handler.setLevel(logging.INFO)

        # Create formatters and add it to handlers
        c_format = logging.Formatter('%(message)s')
        f_format = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
        c_handler.setFormatter(c_format)
        f_handler.setFormatter(f_format)

        # Add handlers to the logger
        self.logger.addHandler(c_handler)
        self.logger.addHandler(f_handler)

    def get_logger(self):
        return self.logger

device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")

#use below for cuda
'''
#load cuda to run best model
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("Running on the GPU: cuda")
else:
    device = torch.device("cpu")
    print("Running on the CPU")
'''

print(f'device: {device}')

device: mps


### B. TRAINING

In [6]:
#one hot encode train_data and drop excess columns
df_encoded = one_hot_encode_multilabels(train_data, 'Labels')
df_encoded_train = df_encoded.drop(columns=['Labels','Caption'])

#train val split
train_proportion = 0.8

dataframe = df_encoded_train
image_dir = 'data/'

# Split the dataset
train_dataset, val_dataset = train_test_split(dataframe, test_size = 1 - train_proportion, random_state = 42)

dataset classses

In [7]:
# Implement transformations on train, val, and test datasets

transform_train = transforms.Compose([
    transforms.Resize((256,256)),
    transforms.RandomCrop((224,224)),
    transforms.ColorJitter(brightness = 0.5),
    transforms.RandomRotation(degrees = 45),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomVerticalFlip(p=0.05),
    transforms.RandomGrayscale(p=0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

transform_val = transforms.Compose([
    transforms.Resize((256,256)),
    transforms.CenterCrop((224,224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

transform_test = transforms.Compose([
    transforms.Resize((256,256)),
    transforms.CenterCrop((224,224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# create train, val, and test datasets
train_dataset = ImageDataset(train_dataset, image_dir, transform=transform_train)
val_dataset = ImageDataset(val_dataset, image_dir, transform=transform_val)
test_dataset = ImageDataset2(dataframe = test_data, image_dir = image_dir, transform=transform_test)

# create train, val and test dataloaders, not generator is set to None
batch_size = 32
generator2 = torch.Generator()
generator2.manual_seed(42)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, generator=None)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, generator=None)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


Define training protocol
- generate_predictions_and_save: generate prediction (csv file) based on trained model and test images
- train_val_run: run training loop for model up to max epoch, while implementing early stopping and dynamic learning rate. This function calls generate_predictions_and_save to create csv output
- run_experiments: implement a series of training by calling train_val_run for different combinations of optimizers.

In [8]:
def generate_predictions_and_save(model,test_data, output_name, output_path = 'models/output/'):
    '''
    Generate prediction (csv file) based on trained model and test images
    
    input: 
    model: pth, trained model
    test_data: df, test data
    output_name: string, name of output file
    output_path: string, path to save output file

    output:
    csv file, submitable to Kaggle
    '''
    #we don't explicitly call model in this function
    model.eval()
    model.to(device)
    
    # Generate prediction
    test_predictions = []
    test_image_ids = test_data['ImageID'].tolist()

    with torch.no_grad():
        for images in test_loader:
            images = images.to(device)
            outputs = model(images)
            predictions = torch.sigmoid(outputs).data > 0.5
            test_predictions.extend(predictions.cpu().numpy())

    # Get label names from df_encoded_train
    label_columns = [col for col in df_encoded_train.columns if col.startswith('Label_')]
    label_names = [col.split('_')[1] for col in label_columns]  # Adjust based on how labels are named

    # Convert predictions to indices
    predicted_labels = [' '.join(label_names[j] for j in range(len(label_names)) if pred[j]) for pred in test_predictions]

    # Combine ImageID and predicted Labels into a DataFrame
    results_df = pd.DataFrame({
        'ImageID': test_image_ids,
        'Labels': predicted_labels
    })

    output_file = f'{output_path}{output_name}.csv'

    # Save as csv
    results_df.to_csv(output_file, index=False)

def train_val_run(model, optimizer, output_name, test_data, epochs, validate_every = 2, lr_scheduler_toggle = True):
    '''
    Run training loop for model up to max epoch, while implementing early stopping and dynamic learning rate. 
    
    input: 
    model: pth, pre-trained model
    optimizer: optimizer to use
    output_name: string, name of output file
    test_data: df, test data
    epochs: integer, max epoch
    validate_every: integer, frequency of validation
    lr_scheduler_toggle: boolean, whether to use dynamic learning rate

    output:
    log file to be saved at designated path
    model pth to be saved at designated path
    csv file to be saved at designated path
    '''
    #extract determined model name and pre-trained model, assign to MPS/CUDA
    model.to(device)

    #setup scheduler
    if lr_scheduler_toggle:
        scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=2)

    #set criterion 
    criterion = torch.nn.BCEWithLogitsLoss()

    #setup loggers
    timestamp = datetime.now().strftime('%Y%m%d%H%M')
    logger = Logger(output_name,timestamp).get_logger()
        
    #record runtimes for stats
    run_time = []

    #implement early stopper to prevent overfitting
    early_stopper = EarlyStopper(patience=6, min_delta=0.001)

    #training and validation loop

    start_epoch = 0

    for epoch in range(start_epoch, epochs):
        start_time = time.time()
        model.train()
        running_loss = 0.0
        current_lr = optimizer.param_groups[0]['lr'] 
        
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        
        epoch_time = time.time() - start_time
        run_time.append(epoch_time)
        logger.info(f'Training   >>> Epoch: {epoch+1}, Current LR: {current_lr}, Loss: {(running_loss/len(train_loader)):.3f}, Time: {int(epoch_time // 60)}m{int(epoch_time % 60)}s')

        # Validation
        if (epoch + 1) % validate_every == 0 or (epoch + 1) == epochs:
            model.eval()
            validation_loss = 0.0
            all_labels = []
            all_predictions = []
            start_time = time.time()

            with torch.no_grad():
                for images, labels in val_loader:
                    images, labels = images.to(device), labels.to(device)
                    outputs = model(images)
                    val_loss = criterion(outputs, labels)
                    validation_loss += val_loss.item()
                    predictions = torch.sigmoid(outputs).data > 0.5  
                    all_labels.append(labels.cpu().numpy())
                    all_predictions.append(predictions.cpu().numpy())

            #calculate validation loss and update scheduler
            validation_loss /= len(val_loader)
            val_time = time.time() - start_time
            run_time.append(val_time)

            #calculate f1 score
            all_labels = np.vstack(all_labels)
            all_predictions = np.vstack(all_predictions)
            f1_scores = f1_score(all_labels, all_predictions, average='macro')

            logger.info(f'Validation >>> Epoch: {epoch+1}, Loss: {(validation_loss):.3f}, current lr: {current_lr}, F1 Score: {f1_scores:.3f}, Time: {int(val_time // 60)}m{int(val_time % 60)}s')

            #updat scheduler
            if lr_scheduler_toggle:
                scheduler.step(validation_loss)

            #early stopper
            if early_stopper.early_stop(validation_loss):
                logger.info(f'Early stopping >>> triggered at epoch {epoch + 1}')
                break

    #total time
    total_time = sum(run_time)
    logger.info(f'Training concluded >>> Total Run Time: {int(total_time // 60)}m{int(total_time % 60)}s')

    #save model and show size

    # Create directory if it doesn't exist
    os.makedirs('models/pth', exist_ok=True)
    path = f'models/pth/{output_name}.pth'
    torch.save(model.state_dict(), path)
    file_size = os.path.getsize(path)
    logger.info(f"Model size on disk: {file_size / (1024 * 1024):.2f} MB")

    #generate prediction output
    generate_predictions_and_save(model,test_data, output_name)

def run_experiments(model_name, model, epochs, optimizers, validate_every = 2):
    '''
    Implement a series of training by calling train_val_run for different combinations of optimizers

    input:
    model_name: string, name of model
    model: pth, pre-trained model
    epochs: integer, max epoch
    optimizers: list of tuples, each tuple contains optimizer name, starting learning rate, and whether to use lr_scheduler
    validate_every: integer, frequency of validation
    '''
    for optimizer in optimizers:
        optimizer_name = optimizer[0]
        optimizer_starting_lr = optimizer[1]
        optimizer_lr_scheduler_toggle = optimizer[2]
        print(f'=========================================================================')
        print(f"Model: {model_name}, optimizer: {optimizer_name}, starting_lr= {optimizer_starting_lr}, lr_scheduler: {optimizer_lr_scheduler_toggle}")
        
        if optimizer_name == 'Adadelta':
            optimizer = torch.optim.Adadelta(model.parameters(), lr=optimizer_starting_lr)
        elif optimizer_name == 'Adam':
            optimizer = torch.optim.Adam(model.parameters(), lr=optimizer_starting_lr) 
        elif optimizer_name == 'SGD':
            optimizer = torch.optim.SGD(model.parameters(), lr=optimizer_starting_lr)

        output_name = f'{model_name}_{optimizer_name}_{optimizer_starting_lr}'
        model = train_val_run(model, optimizer, output_name, test_data, epochs, lr_scheduler_toggle = optimizer_lr_scheduler_toggle)

#### Training protocol

In [7]:
epochs = 60
optimizers = [['Adadelta', 0.01, True],['Adam', 0.01, True],['SGD', 0.01, True]]

ResNext50

In [8]:
model_name = 'resnext50'

# Create DataLoaders for both train and validation sets
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

#run train and val
for optimizer in optimizers:
    optimizer_name = optimizer[0]
    optimizer_starting_lr = optimizer[1]
    optimizer_lr_scheduler_toggle = optimizer[2]
    print(f'============================================+++++++=============================')
    print(f"Model: {model_name}, optimizer: {optimizer_name}, starting_lr= {optimizer_starting_lr}, lr_scheduler: {optimizer_lr_scheduler_toggle}")
    
    #reset model
    model = models.resnext50_32x4d(weights = models.ResNeXt50_32X4D_Weights.DEFAULT)
    model.num_classes = 18
    model.fc = torch.nn.Linear(model.fc.in_features, model.num_classes)  
    
    #call optimizer and starting learning rate:
    if optimizer_name == 'Adadelta':
        optimizer = torch.optim.Adadelta(model.parameters(), lr=optimizer_starting_lr)
    elif optimizer_name == 'Adam':
        optimizer = torch.optim.Adam(model.parameters(), lr=optimizer_starting_lr) 
    elif optimizer_name == 'SGD':
        optimizer = torch.optim.SGD(model.parameters(), lr=optimizer_starting_lr)

    output_name = f'{model_name}_{optimizer_name}_{optimizer_starting_lr}'
    train_val_run(model, optimizer, output_name, test_data, epochs, lr_scheduler_toggle = optimizer_lr_scheduler_toggle)

============================================+++++++=============================
Model: resnext50, optimizer: Adadelta, starting_lr= 0.01, lr_scheduler: True


Training   >>> Epoch: 1, Current LR: 0.01, Loss: 0.271, Time: 8m0s
Training   >>> Epoch: 2, Current LR: 0.01, Loss: 0.182, Time: 7m56s
Validation >>> Epoch: 2, Loss: 0.179, current lr: 0.01, F1 Score: 0.054, Time: 0m40s
Training   >>> Epoch: 3, Current LR: 0.01, Loss: 0.169, Time: 8m3s
Training   >>> Epoch: 4, Current LR: 0.01, Loss: 0.159, Time: 8m4s
Validation >>> Epoch: 4, Loss: 0.155, current lr: 0.01, F1 Score: 0.110, Time: 0m40s
Training   >>> Epoch: 5, Current LR: 0.01, Loss: 0.150, Time: 8m4s
Training   >>> Epoch: 6, Current LR: 0.01, Loss: 0.143, Time: 8m3s
Validation >>> Epoch: 6, Loss: 0.138, current lr: 0.01, F1 Score: 0.241, Time: 0m40s
Training   >>> Epoch: 7, Current LR: 0.01, Loss: 0.137, Time: 8m4s
Training   >>> Epoch: 8, Current LR: 0.01, Loss: 0.132, Time: 8m4s
Validation >>> Epoch: 8, Loss: 0.126, current lr: 0.01, F1 Score: 0.361, Time: 0m40s
Training   >>> Epoch: 9, Current LR: 0.01, Loss: 0.128, Time: 7m58s
Training   >>> Epoch: 10, Current LR: 0.01, Loss: 0.125

============================================+++++++=============================
Model: resnext50, optimizer: Adam, starting_lr= 0.01, lr_scheduler: True


Training   >>> Epoch: 1, Current LR: 0.01, Loss: 0.198, Time: 8m7s
Training   >>> Epoch: 2, Current LR: 0.01, Loss: 0.189, Time: 8m8s
Validation >>> Epoch: 2, Loss: 0.188, current lr: 0.01, F1 Score: 0.048, Time: 0m39s
Training   >>> Epoch: 3, Current LR: 0.01, Loss: 0.187, Time: 8m7s
Training   >>> Epoch: 4, Current LR: 0.01, Loss: 0.186, Time: 8m9s
Validation >>> Epoch: 4, Loss: 0.193, current lr: 0.01, F1 Score: 0.047, Time: 0m39s
Training   >>> Epoch: 5, Current LR: 0.01, Loss: 0.183, Time: 8m4s
Training   >>> Epoch: 6, Current LR: 0.01, Loss: 0.182, Time: 8m6s
Validation >>> Epoch: 6, Loss: 0.188, current lr: 0.01, F1 Score: 0.050, Time: 0m39s
Training   >>> Epoch: 7, Current LR: 0.01, Loss: 0.179, Time: 8m8s
Training   >>> Epoch: 8, Current LR: 0.01, Loss: 0.177, Time: 8m8s
Validation >>> Epoch: 8, Loss: 0.176, current lr: 0.01, F1 Score: 0.089, Time: 0m39s
Training   >>> Epoch: 9, Current LR: 0.01, Loss: 0.175, Time: 8m7s
Training   >>> Epoch: 10, Current LR: 0.01, Loss: 0.173, 

============================================+++++++=============================
Model: resnext50, optimizer: SGD, starting_lr= 0.01, lr_scheduler: True


Training   >>> Epoch: 1, Current LR: 0.01, Loss: 0.258, Time: 7m30s
Training   >>> Epoch: 2, Current LR: 0.01, Loss: 0.179, Time: 7m27s
Validation >>> Epoch: 2, Loss: 0.174, current lr: 0.01, F1 Score: 0.062, Time: 0m40s
Training   >>> Epoch: 3, Current LR: 0.01, Loss: 0.164, Time: 7m27s
Training   >>> Epoch: 4, Current LR: 0.01, Loss: 0.153, Time: 7m29s
Validation >>> Epoch: 4, Loss: 0.147, current lr: 0.01, F1 Score: 0.209, Time: 0m40s
Training   >>> Epoch: 5, Current LR: 0.01, Loss: 0.144, Time: 7m24s
Training   >>> Epoch: 6, Current LR: 0.01, Loss: 0.137, Time: 7m25s
Validation >>> Epoch: 6, Loss: 0.130, current lr: 0.01, F1 Score: 0.342, Time: 0m39s
Training   >>> Epoch: 7, Current LR: 0.01, Loss: 0.131, Time: 7m30s
Training   >>> Epoch: 8, Current LR: 0.01, Loss: 0.127, Time: 7m25s
Validation >>> Epoch: 8, Loss: 0.119, current lr: 0.01, F1 Score: 0.442, Time: 0m39s
Training   >>> Epoch: 9, Current LR: 0.01, Loss: 0.123, Time: 7m24s
Training   >>> Epoch: 10, Current LR: 0.01, Loss

GoogleNet

In [9]:
#download model, assign name, and set variables needed to run for this dataset
model_name = 'googlenet'

# Create DataLoaders for both train and validation sets
batch_size = 128
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

#run train and val   
for optimizer in optimizers:
    optimizer_name = optimizer[0]
    optimizer_starting_lr = optimizer[1]
    optimizer_lr_scheduler_toggle = optimizer[2]
    print(f'============================================+++++++=============================')
    print(f"Model: {model_name}, optimizer: {optimizer_name}, starting_lr= {optimizer_starting_lr}, lr_scheduler: {optimizer_lr_scheduler_toggle}")
    
    #reset model
    model = models.googlenet(weights=models.GoogLeNet_Weights.DEFAULT)
    model.num_classes = 18 
    model.fc = torch.nn.Linear(model.fc.in_features, model.num_classes) 
    
    #call optimizer and starting learning rate:
    if optimizer_name == 'Adadelta':
        optimizer = torch.optim.Adadelta(model.parameters(), lr=optimizer_starting_lr)
    elif optimizer_name == 'Adam':
        optimizer = torch.optim.Adam(model.parameters(), lr=optimizer_starting_lr) 
    elif optimizer_name == 'SGD':
        optimizer = torch.optim.SGD(model.parameters(), lr=optimizer_starting_lr)

    output_name = f'{model_name}_{optimizer_name}_{optimizer_starting_lr}'
    train_val_run(model, optimizer, output_name, test_data, epochs, lr_scheduler_toggle = optimizer_lr_scheduler_toggle)

============================================+++++++=============================
Model: googlenet, optimizer: Adadelta, starting_lr= 0.01, lr_scheduler: True


Training   >>> Epoch: 1, Current LR: 0.01, Loss: 0.419, Time: 3m22s
Training   >>> Epoch: 2, Current LR: 0.01, Loss: 0.227, Time: 4m32s
Validation >>> Epoch: 2, Loss: 0.212, current lr: 0.01, F1 Score: 0.048, Time: 0m19s
Training   >>> Epoch: 3, Current LR: 0.01, Loss: 0.202, Time: 4m23s
Training   >>> Epoch: 4, Current LR: 0.01, Loss: 0.195, Time: 4m25s
Validation >>> Epoch: 4, Loss: 0.193, current lr: 0.01, F1 Score: 0.048, Time: 0m18s
Training   >>> Epoch: 5, Current LR: 0.01, Loss: 0.192, Time: 4m1s
Training   >>> Epoch: 6, Current LR: 0.01, Loss: 0.189, Time: 4m18s
Validation >>> Epoch: 6, Loss: 0.186, current lr: 0.01, F1 Score: 0.048, Time: 0m18s
Training   >>> Epoch: 7, Current LR: 0.01, Loss: 0.187, Time: 4m38s
Training   >>> Epoch: 8, Current LR: 0.01, Loss: 0.185, Time: 4m35s
Validation >>> Epoch: 8, Loss: 0.182, current lr: 0.01, F1 Score: 0.048, Time: 0m18s
Training   >>> Epoch: 9, Current LR: 0.01, Loss: 0.183, Time: 4m36s
Training   >>> Epoch: 10, Current LR: 0.01, Loss:

============================================+++++++=============================
Model: googlenet, optimizer: Adam, starting_lr= 0.01, lr_scheduler: True


Training   >>> Epoch: 1, Current LR: 0.01, Loss: 0.200, Time: 3m43s
Training   >>> Epoch: 2, Current LR: 0.01, Loss: 0.190, Time: 3m46s
Validation >>> Epoch: 2, Loss: 0.188, current lr: 0.01, F1 Score: 0.048, Time: 0m18s
Training   >>> Epoch: 3, Current LR: 0.01, Loss: 0.187, Time: 3m41s
Training   >>> Epoch: 4, Current LR: 0.01, Loss: 0.185, Time: 3m46s
Validation >>> Epoch: 4, Loss: 0.187, current lr: 0.01, F1 Score: 0.046, Time: 0m18s
Training   >>> Epoch: 5, Current LR: 0.01, Loss: 0.182, Time: 3m41s
Training   >>> Epoch: 6, Current LR: 0.01, Loss: 0.179, Time: 3m37s
Validation >>> Epoch: 6, Loss: 0.186, current lr: 0.01, F1 Score: 0.068, Time: 0m18s
Training   >>> Epoch: 7, Current LR: 0.01, Loss: 0.177, Time: 3m41s
Training   >>> Epoch: 8, Current LR: 0.01, Loss: 0.174, Time: 3m46s
Validation >>> Epoch: 8, Loss: 0.187, current lr: 0.01, F1 Score: 0.107, Time: 0m18s
Training   >>> Epoch: 9, Current LR: 0.01, Loss: 0.173, Time: 3m44s
Training   >>> Epoch: 10, Current LR: 0.01, Loss

============================================+++++++=============================
Model: googlenet, optimizer: SGD, starting_lr= 0.01, lr_scheduler: True


Training   >>> Epoch: 1, Current LR: 0.01, Loss: 0.317, Time: 3m9s
Training   >>> Epoch: 2, Current LR: 0.01, Loss: 0.211, Time: 3m12s
Validation >>> Epoch: 2, Loss: 0.204, current lr: 0.01, F1 Score: 0.048, Time: 0m18s
Training   >>> Epoch: 3, Current LR: 0.01, Loss: 0.199, Time: 3m9s
Training   >>> Epoch: 4, Current LR: 0.01, Loss: 0.194, Time: 3m14s
Validation >>> Epoch: 4, Loss: 0.191, current lr: 0.01, F1 Score: 0.048, Time: 0m17s
Training   >>> Epoch: 5, Current LR: 0.01, Loss: 0.190, Time: 3m10s
Training   >>> Epoch: 6, Current LR: 0.01, Loss: 0.188, Time: 3m12s
Validation >>> Epoch: 6, Loss: 0.185, current lr: 0.01, F1 Score: 0.048, Time: 0m18s
Training   >>> Epoch: 7, Current LR: 0.01, Loss: 0.186, Time: 3m12s
Training   >>> Epoch: 8, Current LR: 0.01, Loss: 0.183, Time: 3m10s
Validation >>> Epoch: 8, Loss: 0.181, current lr: 0.01, F1 Score: 0.048, Time: 0m17s
Training   >>> Epoch: 9, Current LR: 0.01, Loss: 0.182, Time: 3m11s
Training   >>> Epoch: 10, Current LR: 0.01, Loss: 

Resnet

In [10]:
model_name = 'resnet50'

# Create DataLoaders for both train and validation sets
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

#run train and val
for optimizer in optimizers:
    optimizer_name = optimizer[0]
    optimizer_starting_lr = optimizer[1]
    optimizer_lr_scheduler_toggle = optimizer[2]
    print(f'============================================+++++++=============================')
    print(f"Model: {model_name}, optimizer: {optimizer_name}, starting_lr= {optimizer_starting_lr}, lr_scheduler: {optimizer_lr_scheduler_toggle}")
    
    #reset model
    model = models.resnet50(weights = models.ResNet50_Weights.DEFAULT)
    model.num_classes = 18
    model.fc = torch.nn.Linear(model.fc.in_features, model.num_classes)  
    
    #call optimizer and starting learning rate:
    if optimizer_name == 'Adadelta':
        optimizer = torch.optim.Adadelta(model.parameters(), lr=optimizer_starting_lr)
    elif optimizer_name == 'Adam':
        optimizer = torch.optim.Adam(model.parameters(), lr=optimizer_starting_lr) 
    elif optimizer_name == 'SGD':
        optimizer = torch.optim.SGD(model.parameters(), lr=optimizer_starting_lr)

    output_name = f'{model_name}_{optimizer_name}_{optimizer_starting_lr}'
    train_val_run(model, optimizer, output_name, test_data, epochs, lr_scheduler_toggle = optimizer_lr_scheduler_toggle)

============================================+++++++=============================
Model: resnet50, optimizer: Adadelta, starting_lr= 0.01, lr_scheduler: True


Training   >>> Epoch: 1, Current LR: 0.01, Loss: 0.292, Time: 6m21s
Training   >>> Epoch: 2, Current LR: 0.01, Loss: 0.183, Time: 6m16s
Validation >>> Epoch: 2, Loss: 0.179, current lr: 0.01, F1 Score: 0.048, Time: 0m33s
Training   >>> Epoch: 3, Current LR: 0.01, Loss: 0.170, Time: 6m17s
Training   >>> Epoch: 4, Current LR: 0.01, Loss: 0.160, Time: 6m17s
Validation >>> Epoch: 4, Loss: 0.154, current lr: 0.01, F1 Score: 0.089, Time: 0m33s
Training   >>> Epoch: 5, Current LR: 0.01, Loss: 0.151, Time: 6m17s
Training   >>> Epoch: 6, Current LR: 0.01, Loss: 0.144, Time: 6m17s
Validation >>> Epoch: 6, Loss: 0.134, current lr: 0.01, F1 Score: 0.242, Time: 0m33s
Training   >>> Epoch: 7, Current LR: 0.01, Loss: 0.138, Time: 6m17s
Training   >>> Epoch: 8, Current LR: 0.01, Loss: 0.132, Time: 6m18s
Validation >>> Epoch: 8, Loss: 0.122, current lr: 0.01, F1 Score: 0.369, Time: 0m33s
Training   >>> Epoch: 9, Current LR: 0.01, Loss: 0.128, Time: 6m16s
Training   >>> Epoch: 10, Current LR: 0.01, Loss

============================================+++++++=============================
Model: resnet50, optimizer: Adam, starting_lr= 0.01, lr_scheduler: True


Training   >>> Epoch: 1, Current LR: 0.01, Loss: 0.198, Time: 6m9s
Training   >>> Epoch: 2, Current LR: 0.01, Loss: 0.190, Time: 6m7s
Validation >>> Epoch: 2, Loss: 0.192, current lr: 0.01, F1 Score: 0.048, Time: 0m32s
Training   >>> Epoch: 3, Current LR: 0.01, Loss: 0.188, Time: 6m10s
Training   >>> Epoch: 4, Current LR: 0.01, Loss: 0.187, Time: 6m9s
Validation >>> Epoch: 4, Loss: 0.192, current lr: 0.01, F1 Score: 0.055, Time: 0m32s
Training   >>> Epoch: 5, Current LR: 0.01, Loss: 0.186, Time: 6m9s
Training   >>> Epoch: 6, Current LR: 0.01, Loss: 0.185, Time: 6m9s
Validation >>> Epoch: 6, Loss: 0.196, current lr: 0.01, F1 Score: 0.058, Time: 0m32s
Training   >>> Epoch: 7, Current LR: 0.01, Loss: 0.184, Time: 6m10s
Training   >>> Epoch: 8, Current LR: 0.01, Loss: 0.182, Time: 6m10s
Validation >>> Epoch: 8, Loss: 0.186, current lr: 0.01, F1 Score: 0.069, Time: 0m33s
Training   >>> Epoch: 9, Current LR: 0.01, Loss: 0.181, Time: 6m10s
Training   >>> Epoch: 10, Current LR: 0.01, Loss: 0.1

============================================+++++++=============================
Model: resnet50, optimizer: SGD, starting_lr= 0.01, lr_scheduler: True


Training   >>> Epoch: 1, Current LR: 0.01, Loss: 0.277, Time: 6m0s
Training   >>> Epoch: 2, Current LR: 0.01, Loss: 0.179, Time: 5m59s
Validation >>> Epoch: 2, Loss: 0.174, current lr: 0.01, F1 Score: 0.054, Time: 0m32s
Training   >>> Epoch: 3, Current LR: 0.01, Loss: 0.165, Time: 6m0s
Training   >>> Epoch: 4, Current LR: 0.01, Loss: 0.153, Time: 6m0s
Validation >>> Epoch: 4, Loss: 0.144, current lr: 0.01, F1 Score: 0.211, Time: 0m33s
Training   >>> Epoch: 5, Current LR: 0.01, Loss: 0.143, Time: 6m0s
Training   >>> Epoch: 6, Current LR: 0.01, Loss: 0.136, Time: 6m0s
Validation >>> Epoch: 6, Loss: 0.126, current lr: 0.01, F1 Score: 0.382, Time: 0m32s
Training   >>> Epoch: 7, Current LR: 0.01, Loss: 0.130, Time: 6m0s
Training   >>> Epoch: 8, Current LR: 0.01, Loss: 0.126, Time: 6m0s
Validation >>> Epoch: 8, Loss: 0.115, current lr: 0.01, F1 Score: 0.471, Time: 0m33s
Training   >>> Epoch: 9, Current LR: 0.01, Loss: 0.122, Time: 6m0s
Training   >>> Epoch: 10, Current LR: 0.01, Loss: 0.119,

RegNet

In [11]:
model_name = 'regnet'

# Create DataLoaders for both train and validation sets
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

#run train and val
for optimizer in optimizers:
    optimizer_name = optimizer[0]
    optimizer_starting_lr = optimizer[1]
    optimizer_lr_scheduler_toggle = optimizer[2]
    print(f'============================================+++++++=============================')
    print(f"Model: {model_name}, optimizer: {optimizer_name}, starting_lr= {optimizer_starting_lr}, lr_scheduler: {optimizer_lr_scheduler_toggle}")
    
    #reset model
    model = models.regnet_x_3_2gf(weights = models.RegNet_X_3_2GF_Weights.DEFAULT)
    model.num_classes = 18
    model.fc = torch.nn.Linear(model.fc.in_features, model.num_classes)  
    
    #call optimizer and starting learning rate:
    if optimizer_name == 'Adadelta':
        optimizer = torch.optim.Adadelta(model.parameters(), lr=optimizer_starting_lr)
    elif optimizer_name == 'Adam':
        optimizer = torch.optim.Adam(model.parameters(), lr=optimizer_starting_lr) 
    elif optimizer_name == 'SGD':
        optimizer = torch.optim.SGD(model.parameters(), lr=optimizer_starting_lr)

    
    output_name = f'{model_name}_{optimizer_name}_{optimizer_starting_lr}'
    train_val_run(model, optimizer, output_name, test_data, epochs, lr_scheduler_toggle = optimizer_lr_scheduler_toggle)

============================================+++++++=============================
Model: regnet, optimizer: Adadelta, starting_lr= 0.01, lr_scheduler: True


Training   >>> Epoch: 1, Current LR: 0.01, Loss: 0.257, Time: 8m29s
Training   >>> Epoch: 2, Current LR: 0.01, Loss: 0.174, Time: 8m16s
Validation >>> Epoch: 2, Loss: 0.170, current lr: 0.01, F1 Score: 0.064, Time: 0m32s
Training   >>> Epoch: 3, Current LR: 0.01, Loss: 0.159, Time: 8m12s
Training   >>> Epoch: 4, Current LR: 0.01, Loss: 0.148, Time: 8m18s
Validation >>> Epoch: 4, Loss: 0.142, current lr: 0.01, F1 Score: 0.237, Time: 0m31s
Training   >>> Epoch: 5, Current LR: 0.01, Loss: 0.139, Time: 8m36s
Training   >>> Epoch: 6, Current LR: 0.01, Loss: 0.133, Time: 8m18s
Validation >>> Epoch: 6, Loss: 0.126, current lr: 0.01, F1 Score: 0.387, Time: 0m31s
Training   >>> Epoch: 7, Current LR: 0.01, Loss: 0.128, Time: 8m21s
Training   >>> Epoch: 8, Current LR: 0.01, Loss: 0.123, Time: 8m17s
Validation >>> Epoch: 8, Loss: 0.115, current lr: 0.01, F1 Score: 0.460, Time: 0m31s
Training   >>> Epoch: 9, Current LR: 0.01, Loss: 0.120, Time: 8m17s
Training   >>> Epoch: 10, Current LR: 0.01, Loss

============================================+++++++=============================
Model: regnet, optimizer: Adam, starting_lr= 0.01, lr_scheduler: True


Training   >>> Epoch: 1, Current LR: 0.01, Loss: 0.205, Time: 8m59s
Training   >>> Epoch: 2, Current LR: 0.01, Loss: 0.193, Time: 8m53s
Validation >>> Epoch: 2, Loss: 0.192, current lr: 0.01, F1 Score: 0.048, Time: 0m31s
Training   >>> Epoch: 3, Current LR: 0.01, Loss: 0.191, Time: 8m52s
Training   >>> Epoch: 4, Current LR: 0.01, Loss: 0.188, Time: 8m57s
Validation >>> Epoch: 4, Loss: 0.188, current lr: 0.01, F1 Score: 0.053, Time: 0m31s
Training   >>> Epoch: 5, Current LR: 0.01, Loss: 0.186, Time: 8m57s
Training   >>> Epoch: 6, Current LR: 0.01, Loss: 0.185, Time: 8m55s
Validation >>> Epoch: 6, Loss: 0.198, current lr: 0.01, F1 Score: 0.093, Time: 0m31s
Training   >>> Epoch: 7, Current LR: 0.01, Loss: 0.183, Time: 8m55s
Training   >>> Epoch: 8, Current LR: 0.01, Loss: 0.182, Time: 8m53s
Validation >>> Epoch: 8, Loss: 0.188, current lr: 0.01, F1 Score: 0.112, Time: 0m31s
Training   >>> Epoch: 9, Current LR: 0.01, Loss: 0.180, Time: 8m52s
Training   >>> Epoch: 10, Current LR: 0.01, Loss

============================================+++++++=============================
Model: regnet, optimizer: SGD, starting_lr= 0.01, lr_scheduler: True


Training   >>> Epoch: 1, Current LR: 0.01, Loss: 0.225, Time: 7m8s
Training   >>> Epoch: 2, Current LR: 0.01, Loss: 0.166, Time: 7m1s
Validation >>> Epoch: 2, Loss: 0.158, current lr: 0.01, F1 Score: 0.132, Time: 0m30s
Training   >>> Epoch: 3, Current LR: 0.01, Loss: 0.149, Time: 7m55s
Training   >>> Epoch: 4, Current LR: 0.01, Loss: 0.138, Time: 7m22s
Validation >>> Epoch: 4, Loss: 0.130, current lr: 0.01, F1 Score: 0.377, Time: 0m30s
Training   >>> Epoch: 5, Current LR: 0.01, Loss: 0.130, Time: 6m59s
Training   >>> Epoch: 6, Current LR: 0.01, Loss: 0.125, Time: 6m37s
Validation >>> Epoch: 6, Loss: 0.115, current lr: 0.01, F1 Score: 0.473, Time: 0m32s
Training   >>> Epoch: 7, Current LR: 0.01, Loss: 0.120, Time: 6m34s
Training   >>> Epoch: 8, Current LR: 0.01, Loss: 0.116, Time: 6m31s
Validation >>> Epoch: 8, Loss: 0.108, current lr: 0.01, F1 Score: 0.527, Time: 0m33s
Training   >>> Epoch: 9, Current LR: 0.01, Loss: 0.114, Time: 6m40s
Training   >>> Epoch: 10, Current LR: 0.01, Loss: 

Shuffenet v2

In [12]:
model_name = 'shufflenetv2'

# Create DataLoaders for both train and validation sets
batch_size = 128
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

#run train and val
for optimizer in optimizers:
    optimizer_name = optimizer[0]
    optimizer_starting_lr = optimizer[1]
    optimizer_lr_scheduler_toggle = optimizer[2]
    print(f'============================================+++++++=============================')
    print(f"Model: {model_name}, optimizer: {optimizer_name}, starting_lr= {optimizer_starting_lr}, lr_scheduler: {optimizer_lr_scheduler_toggle}")
    
    #reset model
    model = models.shufflenet_v2_x2_0(weights=models.ShuffleNet_V2_X2_0_Weights.DEFAULT)
    model.num_classes = 18
    model.fc = torch.nn.Linear(model.fc.in_features, model.num_classes)  
    
    #call optimizer and starting learning rate:
    if optimizer_name == 'Adadelta':
        optimizer = torch.optim.Adadelta(model.parameters(), lr=optimizer_starting_lr)
    elif optimizer_name == 'Adam':
        optimizer = torch.optim.Adam(model.parameters(), lr=optimizer_starting_lr) 
    elif optimizer_name == 'SGD':
        optimizer = torch.optim.SGD(model.parameters(), lr=optimizer_starting_lr)

    output_name = f'{model_name}_{optimizer_name}_{optimizer_starting_lr}'
    train_val_run(model, optimizer, output_name, test_data, epochs, lr_scheduler_toggle = optimizer_lr_scheduler_toggle)

============================================+++++++=============================
Model: shufflenetv2, optimizer: Adadelta, starting_lr= 0.01, lr_scheduler: True


Training   >>> Epoch: 1, Current LR: 0.01, Loss: 0.459, Time: 5m55s
Training   >>> Epoch: 2, Current LR: 0.01, Loss: 0.259, Time: 4m50s
Validation >>> Epoch: 2, Loss: 0.259, current lr: 0.01, F1 Score: 0.048, Time: 0m20s
Training   >>> Epoch: 3, Current LR: 0.01, Loss: 0.215, Time: 5m59s
Training   >>> Epoch: 4, Current LR: 0.01, Loss: 0.199, Time: 6m23s
Validation >>> Epoch: 4, Loss: 0.203, current lr: 0.01, F1 Score: 0.048, Time: 0m20s
Training   >>> Epoch: 5, Current LR: 0.01, Loss: 0.190, Time: 7m7s
Training   >>> Epoch: 6, Current LR: 0.01, Loss: 0.184, Time: 6m50s
Validation >>> Epoch: 6, Loss: 0.187, current lr: 0.01, F1 Score: 0.048, Time: 0m20s
Training   >>> Epoch: 7, Current LR: 0.01, Loss: 0.180, Time: 7m28s
Training   >>> Epoch: 8, Current LR: 0.01, Loss: 0.177, Time: 6m1s
Validation >>> Epoch: 8, Loss: 0.177, current lr: 0.01, F1 Score: 0.048, Time: 0m19s
Training   >>> Epoch: 9, Current LR: 0.01, Loss: 0.174, Time: 6m30s
Training   >>> Epoch: 10, Current LR: 0.01, Loss: 

============================================+++++++=============================
Model: shufflenetv2, optimizer: Adam, starting_lr= 0.01, lr_scheduler: True


Training   >>> Epoch: 1, Current LR: 0.01, Loss: 0.200, Time: 3m51s
Training   >>> Epoch: 2, Current LR: 0.01, Loss: 0.183, Time: 3m50s
Validation >>> Epoch: 2, Loss: 0.180, current lr: 0.01, F1 Score: 0.076, Time: 0m20s
Training   >>> Epoch: 3, Current LR: 0.01, Loss: 0.177, Time: 3m59s
Training   >>> Epoch: 4, Current LR: 0.01, Loss: 0.173, Time: 3m47s
Validation >>> Epoch: 4, Loss: 0.171, current lr: 0.01, F1 Score: 0.158, Time: 0m19s
Training   >>> Epoch: 5, Current LR: 0.01, Loss: 0.169, Time: 3m53s
Training   >>> Epoch: 6, Current LR: 0.01, Loss: 0.167, Time: 3m56s
Validation >>> Epoch: 6, Loss: 0.167, current lr: 0.01, F1 Score: 0.188, Time: 0m19s
Training   >>> Epoch: 7, Current LR: 0.01, Loss: 0.164, Time: 3m53s
Training   >>> Epoch: 8, Current LR: 0.01, Loss: 0.162, Time: 3m48s
Validation >>> Epoch: 8, Loss: 0.164, current lr: 0.01, F1 Score: 0.232, Time: 0m19s
Training   >>> Epoch: 9, Current LR: 0.01, Loss: 0.159, Time: 3m56s
Training   >>> Epoch: 10, Current LR: 0.01, Loss

============================================+++++++=============================
Model: shufflenetv2, optimizer: SGD, starting_lr= 0.01, lr_scheduler: True


Training   >>> Epoch: 1, Current LR: 0.01, Loss: 0.416, Time: 4m22s
Training   >>> Epoch: 2, Current LR: 0.01, Loss: 0.246, Time: 4m9s
Validation >>> Epoch: 2, Loss: 0.248, current lr: 0.01, F1 Score: 0.048, Time: 0m22s
Training   >>> Epoch: 3, Current LR: 0.01, Loss: 0.211, Time: 4m23s
Training   >>> Epoch: 4, Current LR: 0.01, Loss: 0.197, Time: 4m7s
Validation >>> Epoch: 4, Loss: 0.200, current lr: 0.01, F1 Score: 0.048, Time: 0m23s
Training   >>> Epoch: 5, Current LR: 0.01, Loss: 0.189, Time: 4m26s
Training   >>> Epoch: 6, Current LR: 0.01, Loss: 0.183, Time: 4m9s
Validation >>> Epoch: 6, Loss: 0.185, current lr: 0.01, F1 Score: 0.048, Time: 0m22s
Training   >>> Epoch: 7, Current LR: 0.01, Loss: 0.179, Time: 4m23s
Training   >>> Epoch: 8, Current LR: 0.01, Loss: 0.176, Time: 4m14s
Validation >>> Epoch: 8, Loss: 0.176, current lr: 0.01, F1 Score: 0.048, Time: 0m23s
Training   >>> Epoch: 9, Current LR: 0.01, Loss: 0.172, Time: 4m26s
Training   >>> Epoch: 10, Current LR: 0.01, Loss: 0

EfficientNet

In [ ]:
#EfficientNet is commented out since it failed to run 
'''
model_name = 'efficientnet' #do this last, epochs too long. AdaDelta 34mins, Adam 1hour and counting

# Create DataLoaders for both train and validation sets
batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

#run train and val
for optimizer in optimizers:
    optimizer_name = optimizer[0]
    optimizer_starting_lr = optimizer[1]
    optimizer_lr_scheduler_toggle = optimizer[2]
    print(f'============================================+++++++=============================')
    print(f"Model: {model_name}, optimizer: {optimizer_name}, starting_lr= {optimizer_starting_lr}, lr_scheduler: {optimizer_lr_scheduler_toggle}")
    
    #reset model
    model = models.efficientnet_v2_s(weights=models.EfficientNet_V2_S_Weights.DEFAULT)
    model.num_classes = 18
    model.classifier[1] = torch.nn.Linear(1280, model.num_classes)
    
    #call optimizer and starting learning rate:
    if optimizer_name == 'Adadelta':
        optimizer = torch.optim.Adadelta(model.parameters(), lr=optimizer_starting_lr)
    elif optimizer_name == 'Adam':
        optimizer = torch.optim.Adam(model.parameters(), lr=optimizer_starting_lr) 
    elif optimizer_name == 'SGD':
        optimizer = torch.optim.SGD(model.parameters(), lr=optimizer_starting_lr)

    output_name = f'{model_name}_{optimizer_name}_{optimizer_starting_lr}'
    train_val_run(model, optimizer, output_name, test_data, epochs, lr_scheduler_toggle = optimizer_lr_scheduler_toggle)
'''